
# Project 1: Build an LLM Playground

Welcome! In this project, you’ll learn foundations of large language models (LLMs). We’ll keep the code minimal and the explanations high‑level so that anyone who can run a Python cell can follow along.  

We'll be using Google Colab for this project. Colab is a free, browser-based platform that lets you run Python code and machine learning models without installing anything on your local computer. Click the button below to open this notebook directly in Google Colab and get started!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bytebyteai/ai-eng-projects/blob/main/project_1/lm_playground.ipynb)

---
## Learning Objectives  
* **Tokenization** and how raw text is tokenized into a sequene of discrete tokens
* Inspect **GPT2** and **Transformer architecture**
* Loading pre-trained LLMs using **Hugging Face**
* **Decoding strategies** to generate text from LLMs
* Completion versus **intrusction fine-tuned** LLMs


Let's get started!

In [ ]:
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)

torch 2.8.0+cu126 | transformers 4.56.2


# 1 - Tokenization

A neural network can’t digest raw text. They need **numbers**. Tokenization is the process of converting text into IDs. In this section, you'll learn how tokenization is implemented in practice.

Tokenization methods generally fall into three categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word‑level tokenization

Split text on whitespace and store each **word** as a token.

In [ ]:
# 1. Tiny corpus
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# 2. Build the vocabulary
PAD, UNK = "[PAD]", "[UNK]"
PAD_ID = -1
UNK_ID = -2

vocab = []
word2id = {}
id2word = {}

vocab = [word for sentence in corpus for word in sentence.split(" ")]

id_iterator = 0
for word in vocab:
  if word in word2id:
    continue

  word2id[word] = id_iterator
  id2word[id_iterator] = word

  id_iterator += 1

print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:15])

# 3. Encode / decode
def encode_word(word: str):
  if word in word2id:
    return word2id[word]

  if word == " ":
    return PAD_ID

  return UNK_ID

def decode_id(id: int):
  if id in id2word:
    return id2word[id]

  if id == PAD_ID:
    return PAD

  return UNK

def encode(text: str):
    result = []
    for word in text.split(" "):
      result.append(encode_word(word))
    return result

def decode(ids: list[int]):
    result = []
    for id in ids:
      result.append(decode_id(id))
    return result

# 4. Demo
sample = "The brown unicorn jumps"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)

Vocabulary size: 21 words
First 15 vocab entries: ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', 'Tokenization', 'converts', 'text', 'to', 'numbers', 'Large']

Input text : The brown unicorn jumps
Token IDs  : [0, 2, -2, 4]
Decoded    : ['The', 'brown', '[UNK]', 'jumps']


Word-level tokenization has two major limitations:
1. Large vocabulary size
2. Out-of-vocabulary (OOV) issue

### 1.2 - Character‑level tokenization

Every single character (including spaces and emojis) gets its own ID. This guarantees zero out‑of‑vocabulary issues but very long sequences.

In [ ]:
# 1. Build a fixed vocabulary # a–z + A–Z + padding + unkwown
import string

vocab = []
char2id = {}
id2char = {}

PAD, UNK = "[PAD]", "[UNK]"

for i in range(26):
  lower_case_c = chr(ord('a') + i)
  vocab.append(lower_case_c)
  char2id[lower_case_c] = i
  id2char[i] = lower_case_c

  upper_case_c = chr(ord('A') + i)
  vocab.append(upper_case_c)
  char2id[upper_case_c] = 26 + i
  id2char[26 + i] = upper_case_c

UNK_ID = 52
vocab.append(UNK)
char2id[UNK] = UNK_ID
id2char[UNK_ID] = UNK

PAD_ID = 53
vocab.append(PAD)
char2id[PAD] = PAD_ID
id2char[PAD_ID] = PAD

print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")
print(f"char2id: {char2id}")
print(f"id2char: {id2char}")

# 2. Encode / decode
def encode_char(char: str):
  if char in char2id:
    return char2id[char]

  if char == " ":
    return PAD_ID

  return UNK_ID

def decode_id(id: int):
  if id in id2char:
    return id2char[id]

  if id == PAD_ID:
    return PAD

  return UNK

def encode(text: str):
    result = []
    for char in list(text):
      result.append(encode_char(char))
    return result

def decode(ids: list[int]):
    result = []
    for id in ids:
      result.append(decode_id(id))
    return result

# 3. Demo
sample = "Hello"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)


Vocabulary size: 54 (52 letters + 2 specials)
char2id: {'a': 0, 'A': 26, 'b': 1, 'B': 27, 'c': 2, 'C': 28, 'd': 3, 'D': 29, 'e': 4, 'E': 30, 'f': 5, 'F': 31, 'g': 6, 'G': 32, 'h': 7, 'H': 33, 'i': 8, 'I': 34, 'j': 9, 'J': 35, 'k': 10, 'K': 36, 'l': 11, 'L': 37, 'm': 12, 'M': 38, 'n': 13, 'N': 39, 'o': 14, 'O': 40, 'p': 15, 'P': 41, 'q': 16, 'Q': 42, 'r': 17, 'R': 43, 's': 18, 'S': 44, 't': 19, 'T': 45, 'u': 20, 'U': 46, 'v': 21, 'V': 47, 'w': 22, 'W': 48, 'x': 23, 'X': 49, 'y': 24, 'Y': 50, 'z': 25, 'Z': 51, '[UNK]': 52, '[PAD]': 53}
id2char: {0: 'a', 26: 'A', 1: 'b', 27: 'B', 2: 'c', 28: 'C', 3: 'd', 29: 'D', 4: 'e', 30: 'E', 5: 'f', 31: 'F', 6: 'g', 32: 'G', 7: 'h', 33: 'H', 8: 'i', 34: 'I', 9: 'j', 35: 'J', 10: 'k', 36: 'K', 11: 'l', 37: 'L', 12: 'm', 38: 'M', 13: 'n', 39: 'N', 14: 'o', 40: 'O', 15: 'p', 41: 'P', 16: 'q', 42: 'Q', 17: 'r', 43: 'R', 18: 's', 44: 'S', 19: 't', 45: 'T', 20: 'u', 46: 'U', 21: 'v', 47: 'V', 22: 'w', 48: 'W', 23: 'x', 49: 'X', 24: 'y', 50: 'Y', 25: 'z', 5

### 1.3 - Subword‑level tokenization

Sub-word methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** the most common character and group them into new tokens. For example, the word `unbelievable` might turn into three tokens: `["un", "believ", "able"]`. This approach strikes a balance between word-level and character-level methods and fix their limitations.

For example, `BPE` algorithm forms the vocabulary using the following steps:
1. **Start with bytes** → every character is its own token.  
2. **Count all adjacent pairs** in a huge corpus.  
3. **Merge the most frequent pair** into a new token.  
   *Repeat steps 2-3* until you hit the target vocab size (e.g., 50 k).

Let's see `BPE` in practice.

In [ ]:
# 1. Load a pretrained BPE tokenizer (GPT-2 uses BPE).
# Refer to  https://huggingface.co/docs/transformers/en/fast_tokenizers

from transformers import AutoTokenizer
from tokenizers import Tokenizer
from tokenizers.models import BPE

bpe_tok = AutoTokenizer.from_pretrained("openai-community/gpt2")

# 2. Encode / decode
def encode(text):
    tokens = bpe_tok.tokenize(text)
    return bpe_tok.convert_tokens_to_ids(tokens)


def decode(ids):
    return bpe_tok.decode(ids)

# 3. Demo
sample = "Unbelievable tokenization powers! 🚀"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Tokens     :", bpe_tok.convert_ids_to_tokens(ids))
print("Decoded    :", recovered)



Input text : Unbelievable tokenization powers! 🚀
Token IDs  : [3118, 6667, 11203, 540, 11241, 1634, 5635, 0, 12520, 248, 222]
Tokens     : ['Un', 'bel', 'iev', 'able', 'Ġtoken', 'ization', 'Ġpowers', '!', 'ĠðŁ', 'ļ', 'Ģ']
Decoded    : Unbelievable tokenization powers! 🚀


### 1.4 - TikToken

`tiktoken` is a production-ready library which offers high‑speed tokenization used by OpenAI models.  
Let's compare the older **gpt2** encoding with the newer **cl100k_base** used in GPT‑4.

In [ ]:
# Use gpt2 and cl100k_base to encode and decode the following text
# Refer to https://github.com/openai/tiktoken
import tiktoken

sentence = "The 🌟 star-player scored 40 points!"

encoding = enc = tiktoken.get_encoding("o200k_base")
encoded = encoding.encode(sentence)
decoded = encoding.decode(encoded)

print("\nInput text :", sentence)
print("Token IDs  :", encoded)
print("Decoded    :", decoded)


Input text : The 🌟 star-player scored 40 points!
Token IDs  : [976, 130321, 253, 8253, 92135, 27207, 220, 1723, 5571, 0]
Decoded    : The 🌟 star-player scored 40 points!


Experiment: try new sentences, emojis, code snippets, or other languages. If you are interested, try implementing the BPE algorithm yourself.

### 1.5 - Key Takeaways

* **Word‑level**: simple but brittle (OOV problems).  
* **Character‑level**: robust but produces long sequences.  
* **BPE / Byte‑Level BPE**: middle ground used by most LLMs.  
* **tiktoken**: shows how production models tokenize with pre‑trained sub‑word vocabularies.

# 2. What is a Language Model?

At its core, a **language model (LM)** is just a *very large* mathematical function built from many neural-network layers.  
Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it learns to output a probability for the next token `tₙ₊₁`.


Each layer applies a simple operation (matrix multiplication, attention, etc.). Stacking hundreds of these layers lets the model capture patterns and statistical relations from text. The final output is a vector of scores that says, “how likely is each possible token to come next?”

> Think of the whole network as **one gigantic equation** whose parameters were tuned during training to minimize prediction error.



### 2.1 - A Single `Linear` Layer

Before we explore Transformer, let’s start tiny:

* A **Linear layer** performs `y = Wx + b`  
  * `x` – input vector  
  * `W` – weight matrix (learned)  
  * `b` – bias vector (learned)

Although this looks basic, chaining thousands of such linear transforms (with nonlinearities in between) gives neural nets their expressive power.


In [ ]:
import torch.nn as nn
class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        # init wieghts and biases
        self.wieght = nn.Parameter(torch.randn(out_features, in_features) * 0.01)
        self.bias = nn.Parameter(torch.zeros(out_features))

    def forward(self, x):
        return x @ self.wieght.T + self.bias

In [ ]:
import torch.nn as nn, torch

lin = nn.Linear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


Input : tensor([ 1.0000, -1.0000,  0.5000])
Weights: Parameter containing:
tensor([[-0.4236, -0.5265, -0.5709],
        [ 0.2812, -0.2966,  0.4482]], requires_grad=True)
Bias   : Parameter containing:
tensor([-0.3767, -0.2979], requires_grad=True)
Output : tensor([-0.5592,  0.5041], grad_fn=<ViewBackward0>)


### 2.2 - A `Transformer` Layer

Most LLMs are a **stack of identical Transformer blocks**. Each block fuses two main components:

| Step | What it does | Where it lives in code |
|------|--------------|------------------------|
| **Multi-Head Self-Attention** | Every token looks at every other token and decides *what matters*. | `block.attn` |
| **Feed-Forward Network (MLP)** | Re-mixes information token-by-token. | `block.mlp` |

Below, we load the smallest public GPT-2 (124 M parameters), grab its *first* block, and inspect the pieces.


In [ ]:
import torch
from transformers import GPT2LMHeadModel

# Load the 124 M-parameter GPT-2 and inspect its layers (12 layers)
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Inspect model structure
print("model.transformer :", model.transformer)             # Shows the backbone
print("model.transformer.h :", model.transformer.h)          # This is a ModuleList of 12 blocks
print("len(model.transformer.h) :", len(model.transformer.h))     # Should print: 12

first_block = model.transformer.h[0]
# This is the first layer of GPT-2, which contains:
print("first_block: ", first_block)

# Self-attention (attn)
print("first_block.attn: ", first_block.attn)     # Multi-head self-attention
# Feedforward MLP (mlp)
print("first_block.mlp: ", first_block.mlp)      # Feedforward network
# LayerNorms (ln_1, ln_2)
print("first_block.ln_1: ", first_block.ln_1)     # LayerNorm before attention
print("first_block.ln_2: ", first_block.ln_2)     # LayerNorm before MLP

model.transformer : GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)
model.transformer.h : ModuleList(
  (0-11): 12 x GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D(nf=2304,

In [ ]:
# Run a tiny forward pass through the first block
seq_len = 8
dummy_tokens = torch.randint(0, model.config.vocab_size, (1, seq_len))
with torch.no_grad():
    # Embed tokens + positions the same way GPT-2 does
    # Forward through one layer

    # Get embeddings first
    hidden_states = model.transformer.wte(dummy_tokens) + model.transformer.wpe(torch.arange(dummy_tokens.size(1)))

    # Pass through only the first layer
    output = first_block(hidden_states)
    hidden_states_after = output[0]

    print("\nOutput shape :", hidden_states_after.shape) # (batch, seq_len, hidden_size)


Output shape : torch.Size([1, 8, 768])


### 2.3 - Inside GPT-2

GPT-2 is just many of those modules arranged in a repeating *block*. Let's print the modules inside the Transformer.

In [ ]:
# Print the name and modules inside gpt2
print("modules :", model.transformer.h)

modules : ModuleList(
  (0-11): 12 x GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D(nf=2304, nx=768)
      (c_proj): Conv1D(nf=768, nx=768)
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D(nf=3072, nx=768)
      (c_proj): Conv1D(nf=768, nx=3072)
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)


As you can see, the Transformer holds various modules, arranged from a list of blocks (`h`). The following table summarizes these modules:

| Step | What it does | Why it matters |
|------|--------------|----------------|
| **Token → Embedding** | Converts IDs to vectors | Gives the model a numeric “handle” on words |
| **Positional Encoding** | Adds “where am I?” info | Order matters in language |
| **Multi-Head Self-Attention** | Each token asks “which other tokens should I look at?” | Lets the model relate words across a sentence |
| **Feed-Forward Network** | Two stacked Linear layers with a non-linearity | Mixes information and adds depth |
| **LayerNorm & Residual** | Stabilize training and help gradients flow | Keeps very deep networks trainable |


### 2.4 LLM's output

Passing a token sequence through an **LLM** yields a tensor of **logits** with shape  
`(batch_size, seq_len, vocab_size)`.  
Applying `softmax` on the last dimension turns those logits into probabilities.

The cell below feeds an 8-token dummy sequence, prints the logits shape, and shows the five most likely next tokens for the final position.


In [ ]:
import torch, torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Load gpt2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# Tokenize input text
text = "Hello my name"
encoded = tokenizer(text, return_tensors="pt")

# Get logits by passing the ids to the gpt2 model.
with torch.no_grad():
    out = model(**encoded, return_dict=True)

logits = out.logits                                # [batch, seq_len, vocab]

print("Logits shape :", logits.shape)

# Predict next token
next_token_logits = logits[:, -1, :]                  # [batch, vocab]
probs = torch.softmax(next_token_logits, dim=-1)      # [batch, vocab]

top_p = torch.topk(probs, k=5)
top_ids = top_p.indices[0].tolist()
top_probs = top_p.values[0].tolist()
top_tokens = tokenizer.convert_ids_to_tokens(top_ids)

print("Top-5 predictions for the next token:")
for tok, p, tid in zip(top_tokens, top_probs, top_ids):
    print(f"  {tok!r:>12}  (id={tid})  p={p:.5f}")


Logits shape : torch.Size([1, 3, 50257])
Top-5 predictions for the next token:
         'Ġis'  (id=318)  p=0.77733
           ','  (id=11)  p=0.03734
          "'s"  (id=338)  p=0.03316
        'Ġwas'  (id=373)  p=0.01270
        'Ġand'  (id=290)  p=0.00757


### 2.5 - Key Takeaway

A language model is nothing mystical: it’s a *huge composition* of small, understandable layers trained to predict the next token in a sequence of tokens.

# 3 - Generation
Once an LLM is trained to predict the probabilities, we can generate text from the model. This process is called decoding or sampling.

At each step, the LLM outputs a **probability distribution** over the next token. It is the job of the decoding algorithm to pick the next token, and move on to the next token. There are different decoding algorithms and hyper-parameters to control the generaiton:
* **Greedy** → pick the single highest‑probability token each step (safe but repetitive).  
* **Top‑k / Nucleus (top‑p)** → sample from a subset of likely tokens (adds variety).
* **beam** -> applies beam search to pick tokens
* **Temperature** → a *creativity* knob. Higher values flatten the probability distribution.

### 3.1 - Greedy decoding

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
}
tokenizers, models = {}, {}
# Load models and tokenizers
for k, v in MODELS.items():
  models[k] = AutoModelForCausalLM.from_pretrained(v)
  tokenizers[k] = AutoTokenizer.from_pretrained(v)

def generate(model_key, prompt, strategy="greedy", max_new_tokens=100):
    tok, mdl = tokenizers[model_key], models[model_key]
    inputs = tok(prompt, return_tensors="pt")

    # Keep it simple: CPU by default; if you move model to CUDA, also move inputs
    inputs = {k: v.to(mdl.device) for k, v in inputs.items()}

    # Return the generations based on the provided strategy: greedy, top_k, top_p
    with torch.no_grad():
        if strategy == "greedy":
            output_ids = mdl.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=max_new_tokens,
                do_sample=False,                         # <---- KEY: greedy = no sampling
                eos_token_id=tok.eos_token_id,
                pad_token_id=tok.pad_token_id
            )

        elif strategy == "top-k":
            top_k = 5
            temperature = 1

            output_ids = mdl.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=max_new_tokens,
                do_sample=True,                          # <---- sampling ON
                top_k=int(top_k),                        # <---- limit to top-k tokens
                temperature=float(temperature),          # <---- scale logits
                eos_token_id=tok.eos_token_id,
                pad_token_id=tok.pad_token_id
            )

        elif strategy == "top-p":
            top_p = 0.8
            temperature = 1

            output_ids = mdl.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=max_new_tokens,
                do_sample=True,                          # <---- sampling ON
                top_p=float(top_p),                      # <---- nucleus sampling
                temperature=float(temperature),
                eos_token_id=tok.eos_token_id,
                pad_token_id=tok.pad_token_id
            )

        else:
            raise ValueError(f"Unknown strategy: {strategy}. Use 'greedy', 'top_k', or 'top_p'.")

    return tok.decode(output_ids[0], skip_special_tokens=True)



In [ ]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Greedy ==")
    print(generate("gpt2", prompt, "greedy", 80))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



== GPT-2 | Greedy ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== GPT-2 | Greedy ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 2+2?

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

== GPT-2 | Greedy ==
Suggest a party theme.

The party theme is a simple, simple, and fun way to get your friends to join you.

The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple, simple, and fun way to get your friends



Naively picking the single best token every time has the following issues in practice:

* **Loop**: “The cat is is is…”  
* **Miss long-term payoff**: the highest-probability word *now* might paint you into a boring corner later.

### 3.2 - Top-k or top-p sampling

In [ ]:

tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Top-p ==")
    print(generate("gpt2", prompt, "top-p", 40))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



== GPT-2 | Top-p ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time in the past, when we were in the wilderness, we were all slaves to a man who had given up his own life to serve a great God, and who would never forget to say that he

== GPT-2 | Top-p ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 2+2?

3. A. A. A+2.

4. A. A+1.

5. A. A+2.

6. A. A+

== GPT-2 | Top-p ==
Suggest a party theme. I like to use a theme that is inspired by the music from the movie.

How to play:

The game is simple. You can only play as a hero, or as a


### 3.3 - Try It Yourself

1. Scroll to the list called `tests`.
2. Swap in your own prompts or tweak the decoding strategy.  
3. Re‑run the cell and compare the vibes.

> **Tip:** Try the same prompt with `greedy` vs. `top_p` (0.9) and see how the tone changes. Notice especially how small temperature tweaks can soften or sharpen the prose.

* `strategy`: `"greedy"`, `"beam"`, `"top_k"`, `"top_p"`  
* `temperature`: `0.2 – 2.0`  
* `k` or `p` thresholds



# 4 - Completion vs. Instruction-tuned LLMs

We have seen that we can use GPT2 model to pass an input text and generate a new text. However, this model only continues the provided text. It is not engaging in a dialouge-like conversation and cannot be helpful by answering instructions. On the other hand, **instruction-tuned LLMs** like `Qwen-Chat` go through an extra training stage called **post-training** after the base “completion” model is finished. Because of post-training step, an instruction-tuned LLM will:

* **Read the entire prompt as a request,** not just as text to mimic.  
* **Stay in dialogue mode**. Answer questions, follow steps, ask clarifying queries.  
* **Refuse or safe-complete** when instructions are unsafe or disallowed.  
* **Adopt a consistent persona** (e.g., “Assistant”) rather than drifting into story continuation.


### 4.1 - Qwen1.5-8B vs. GPT2

In the code below we’ll feed the same prompt to:

* **GPT-2 (completion-only)** – it will simply keep writing in the same style.  
* **Qwen-Chat (instruction-tuned)** – it will obey the instruction and respond directly.

Comparing the two outputs makes the difference easy to see.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
    "qwen": "Qwen/Qwen1.5-1.8B-Chat"
}
tokenizers, models = {}, {}
# Load models and tokenizers
for k, v in MODELS.items():
  models[k] = AutoModelForCausalLM.from_pretrained(v)
  tokenizers[k] = AutoTokenizer.from_pretrained(v)


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]


We downloaded two tiny checkpoints: `GPT‑2` (124 M parameters) and `Qwen‑1.5‑Chat` (1.8 B). If the cell took a while, that was mostly network time. Models are stored locally after the first run.

Let's now generate text and compare two models.


In [ ]:

tests=[("Once upon a time","greedy"),("What is 2+2?","top-k"),("Suggest a party theme.","top-p")]
for prompt,strategy in tests:
    for key in ["gpt2","qwen"]:
        print(f"\n== {key.upper()} | {strategy} ==")
        print(generate(key,prompt,strategy,80))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



== GPT2 | greedy ==
Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== QWEN | greedy ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, there was a young girl named Lily who lived in a small village nestled among the rolling hills of her home. Lily had always been fascinated by nature and spent most of her days exploring the surrounding forests, streams, and meadows. She loved to sing songs about the birds that sang in the trees, the flowers that bloomed in the fields, and the animals that roamed free.
One day

== GPT2 | top-k ==
What is 2+2?

This is a simple question, and I've tried to explain it in a bit. The answer is simple: you don't have to think about it.

What is a 2+2?

This means: if you want to know the answer, you need to know 2+2's, and then you need to know what they mean.

What are they?

== QWEN | top-k ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 2+2? 2 + 2 equals 4. 
The answer is 4.不断地尝试，不断地探索，不断的学习和成长。这就像我们的人生旅程一样，充满了未知与挑战，但也充满了机遇和可能。
在学习的过程中，我们会遇到各种各样的困难和问题。有时候，我们需要面对的是一道复杂的数学题，需要花费大量的时间和精力去理解和解答；有时候

== GPT2 | top-p ==
Suggest a party theme.

If the theme is a music project, you can include it in your post.

Make sure to include the theme if it is not part of the app, and also add it to the project.

Use the themes and project tags to add a theme.

If you do not want to use theme names in the app, you can also use a different theme name in

== QWEN | top-p ==
Suggest a party theme. Sure, here's an idea for a party theme that is perfect for a 1920s-inspired event:
"Roaring Twenties Romance Party"
The theme of the party will transport guests back to the roaring days of the Jazz Age with its vibrant colors, bold patterns, and glamorous atmosphere. Here are some elements to include in your party:
- Decorations: Use flapper dresses,


# 5. (Optional) A Small LLM Playground

### 5.1 ‑ Interactive Playground

Enter a prompt, pick a model and decoding strategy, adjust the temperature, and press **Generate** to watch the model respond.


In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Make sure models and tokenizers are loaded
try:
    tokenizers
    models
except NameError:
    raise RuntimeError("Please run the earlier setup cells that load the models before using the playground.")

def generate_playground(model_key, prompt, strategy="greedy", temperature=1.0, max_new_tokens=100):
    tok, mdl = tokenizers[model_key], models[model_key]
    enc = tok(prompt, return_tensors="pt").to(mdl.device)
    gen_args = dict(**enc, max_new_tokens=max_new_tokens, pad_token_id=tok.pad_token_id)
    if strategy == "greedy":
        gen_args["do_sample"] = False
    elif strategy == "top_k":
        gen_args.update(dict(do_sample=True, top_k=50, temperature=temperature))
    elif strategy == "top_p":
        gen_args.update(dict(do_sample=True, top_p=0.9, temperature=temperature))
    else:
        raise ValueError("Unknown strategy")
    out = mdl.generate(**gen_args)
    return tok.decode(out[0], skip_special_tokens=True)

prompt_box = widgets.Textarea(
    value="Tell me a fun fact about space.",
    placeholder="Type your prompt here",
    description="Prompt:",
    layout=widgets.Layout(width="100%", height="120px")
)

model_dropdown = widgets.Dropdown(
    options=[("GPT‑2", "gpt2"), ("Qwen‑1.5‑Chat", "qwen")],
    value="gpt2",
    description="Model:"
)

strategy_dropdown = widgets.Dropdown(
    options=[("Greedy", "greedy"), ("Top‑k", "top_k"), ("Top‑p", "top_p")],
    value="greedy",
    description="Strategy:"
)

temperature_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=2.0,
    step=0.1,
    description="Temp:"
)

generate_button = widgets.Button(description="Generate", button_style="primary")
output_area = widgets.Output()

def on_generate(_):
    output_area.clear_output()
    with output_area:
        try:
            result = generate_playground(
                model_dropdown.value,
                prompt_box.value,
                strategy_dropdown.value,
                temperature_slider.value
            )
            display(Markdown(f"**Output:**\n\n{result}"))
        except Exception as e:
            print("Error:", e)

generate_button.on_click(on_generate)

ui = widgets.VBox([
    prompt_box,
    widgets.HBox([model_dropdown, strategy_dropdown, temperature_slider]),
    generate_button,
    output_area
])

display(ui)



## 🎉 Congratulations!

You’ve just learned, explored, and inspected a real **LLM**. In one project you:
* Learned how **tokenization** works in practice
* Used `tiktoken` library to load and experiment with most advanced tokenizers.
* Explored LLM architecture and inspected GPT2 blocks and layers
* Learned decoding strategies and used `top-p` to generate text from GPT2
* Loaded a powerful chat model, `Qwen1.5-8B` and generated text
* Built an LLM playground


👏 **Great job!** Take a moment to celebrate. You now have a working mental model of how LLMs work. The skills you used here power most LLMs you see everywhere.
